# Overview
The following notebook will import data from AWS S3, look for images (PNG and JPEG) then attempt to label the images for analysis
The notebook requires access to 
* AWS S3
* AWS Rekognition


## Imports and Variables

In [2]:
import base64
import csv
import io
import json
import os
import pathlib
from io import BytesIO

import boto3
import numpy as np
import pandas as pd
import requests
from IPython.display import HTML, display
from PIL import Image

bucket = 'agd-000001-grpdrv-poc'


## Main Function

In [ ]:
def main(bucket):
    transfer_s3_files(source_bucketname, dest_bucketname)

## Function for getting AWS S3 objects and processing

In [ ]:
def list_s3_files(bucket, dest_bucket):
    s3 = boto3.client("s3")
    paginator = s3.get_paginator("list_objects_v2")
    pages = paginator.paginate(Bucket=bucket, Prefix="")
    with open("filetags.csv", "w", newline="") as myfile:
        writer = csv.writer(myfile)
        # df = pd.DataFrame()
        for page in pages:
            for obj in page["Contents"]:
                file_extension = pathlib.Path(obj["Key"]).suffix
                if file_extension == ".jpeg" or file_extension == ".png":
                    base64 = getbase64(obj, bucket)
                    #   print(base64)
                    print("moving file " + obj["Key"])
                    list = [obj["Key"], base64]
                    # s3copy(obj['Key'], bucket, dest_bucket)
                    # copy has been commented out for the moment
                    response = detect_labels(obj["Key"], bucket)
                    for label in response["Labels"]:
                        list.append(label["Name"])
                        list.append(label["Confidence"])
                    print(list)
                    writer.writerow(list)

## Processing

In [ ]:
def list_s3_files(bucket, dest_bucket):
    s3 = boto3.client("s3")
    paginator = s3.get_paginator("list_objects_v2")
    pages = paginator.paginate(Bucket=bucket, Prefix="")
    with open("filetags.csv", "w", newline="") as myfile:
        writer = csv.writer(myfile)
        # df = pd.DataFrame()
        for page in pages:
            for obj in page["Contents"]:
                file_extension = pathlib.Path(obj["Key"]).suffix
                if file_extension == ".jpeg" or file_extension == ".png":
                    base64 = getbase64(obj, bucket)
                    #   print(base64)
                    print("moving file " + obj["Key"])
                    list = [obj["Key"], base64]
                    # s3copy(obj['Key'], bucket, dest_bucket)
                    # copy has been commented out for the moment
                    response = detect_labels(obj["Key"], bucket)
                    for label in response["Labels"]:
                        list.append(label["Name"])
                        list.append(label["Confidence"])
                    print(list)
                    writer.writerow(list)

## Create Function to Tag Images

In [63]:
def detect_labels(photo, bucket):

    client = boto3.client("rekognition")

    response = client.detect_labels(
        Image={"S3Object": {"Bucket": bucket, "Name": photo}}, MaxLabels=10
    )

    print("Detected labels for " + photo)
    print()
    for label in response["Labels"]:
        print("Label: " + label["Name"])
        print("Confidence: " + str(label["Confidence"]))
        print("Instances:")
        for instance in label["Instances"]:
            print("  Bounding box")
            print("    Top: " + str(instance["BoundingBox"]["Top"]))
            print("    Left: " + str(instance["BoundingBox"]["Left"]))
            print("    Width: " + str(instance["BoundingBox"]["Width"]))
            print("    Height: " + str(instance["BoundingBox"]["Height"]))
            print("  Confidence: " + str(instance["Confidence"]))
            print()

        print("Parents:")
        for parent in label["Parents"]:
            print("   " + parent["Name"])
        print("----------")
        print()
    return response

## Function to create Base64 strings from images

In [2]:
def getbase64(obj, bucket):
    # import file
    s3 = boto3.resource("s3")
    file = s3.Object(bucket, obj)
    file = file.get()
    data = file["Body"].read()

    # read and resize 
    img = cv2.imdecode(np.asarray(bytearray(data)), cv2.IMREAD_COLOR)
    scale_percent = 10 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    # to base64
    _, buffer_img= cv2.imencode('.jpg', img)
    b64 = str(base64.b64encode(buffer_img).decode("ascii"))
    
    return b64

## Run

In [ ]:
main()